In [25]:
import pandas as pd
import re

In [26]:
df = pd.read_csv("./pure-methods-jitsi-videobridge.csv")
df.shape

(105, 6)

In [27]:
def extract_from_tags(tag, tags):
    search_string = tag + "=[\w]+"
    extracted_tag = re.findall(search_string, tags)[0]
    extracted_tag = re.findall("\=(.*)", extracted_tag)[0]
    if extracted_tag == "true":
        return True
    else:
        return False

In [28]:
cols = ["visibility", "parent-FQN", "method-name", "param-list", "return-type", 
        "local-variables", "conditionals", "multiple-statements", "loops", "parameters", "returns",
        "switches", "ifs", "static", "returns-primitives"]
final_df = pd.DataFrame(columns = cols)
for index, row in df.iterrows():
    final_df.loc[index, 'visibility'] = row['visibility']
    final_df.loc[index, 'parent-FQN'] = row['parent-FQN']
    final_df.loc[index, 'method-name'] = row['method-name']
    final_df.loc[index, 'param-list'] = row['param-list'].lstrip('[').rsplit(']', 1)[0]
    final_df.loc[index, 'return-type'] = row['return-type']
    final_df.loc[index, 'local-variables'] = extract_from_tags("local_variables", str(row['tags']))
    final_df.loc[index, 'conditionals'] = extract_from_tags("conditionals", str(row['tags']))
    final_df.loc[index, 'multiple-statements'] = extract_from_tags("multiple_statements", str(row['tags']))
    final_df.loc[index, 'loops'] = extract_from_tags("loops", str(row['tags']))
    final_df.loc[index, 'parameters'] = extract_from_tags("parameters", str(row['tags']))
    final_df.loc[index, 'returns'] = extract_from_tags("returns", str(row['tags']))
    final_df.loc[index, 'switches'] = extract_from_tags("switches", str(row['tags']))
    final_df.loc[index, 'ifs'] = extract_from_tags("ifs", str(row['tags']))
    final_df.loc[index, 'static'] = extract_from_tags("static", str(row['tags']))
    final_df.loc[index, 'returns-primitives'] = extract_from_tags("returns_primitives", str(row['tags']))

final_df.head()

,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
0,NaN,org.jitsi.videobridge.cc.vp8.VP8Frame,getMaxSequenceNumber,,int,False,False,False,False,False,True,False,False,False,True
1,public,org.jitsi.videobridge.Conference,getGid,,java.lang.String,False,False,False,False,False,True,False,False,False,False
2,NaN,org.jitsi.videobridge.cc.vp8.VP8Frame,needsKeyframe,,boolean,False,False,False,False,False,True,False,False,False,True
3,NaN,org.jitsi.videobridge.cc.vp8.VP8QualityFilter,needsKeyframe,,boolean,False,False,False,False,False,True,False,False,False,True
4,public,org.jitsi.videobridge.cc.GenericAdaptiveTrackP...,needsKeyframe,,boolean,False,False,False,False,False,True,False,False,False,True


### Methods that return objects

In [29]:
(final_df["returns-primitives"] == False).sum()

57

### Methods that accept parameters

In [30]:
(final_df["parameters"] == True).sum()

16

### Methods with if statements

In [31]:
(final_df["ifs"] == True).sum()

4

### Methods wtih conditionals ?:

In [32]:
(final_df["conditionals"] == True).sum()

4

### Methods with local variables

In [33]:
(final_df["local-variables"] == True).sum()

2

### Methods with multiple statements

In [34]:
(final_df["multiple-statements"] == True).sum()

4

In [35]:
instrumentation_candidates_df = pd.DataFrame(columns = cols)
instrumentation_candidates_df = final_df[((final_df['multiple-statements'] == True) | 
                                          (final_df['ifs'] == True) |
                                          (final_df['conditionals'] == True) | 
                                          (final_df['parameters'] == True) | 
                                          (final_df['switches'] == True) |
                                          (final_df['loops'] == True) |
                                          (final_df['local-variables'] == True)) &
                                          (final_df['static'] == False)]
print(instrumentation_candidates_df.shape)
instrumentation_candidates_df.to_csv(r'./instrumentation-candidates-jitsi.csv', index=False)
instrumentation_candidates_df.tail(60)

(10, 15)


,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
6,public,org.jitsi.videobridge.Conference$NoOpDiagnosti...,put,"java.lang.String, java.lang.Object",java.lang.Object,False,False,False,False,True,True,False,False,False,False
7,public,org.jitsi.videobridge.Conference$NoOpTimeSerie...,put,"java.lang.String, java.lang.Object",java.lang.Object,False,False,False,False,True,True,False,False,False,False
9,public,org.jitsi.videobridge.cc.GenericAdaptiveTrackP...,rewriteRtcp,org.jitsi.rtp.rtcp.RtcpSrPacket,boolean,False,False,False,False,True,True,False,False,False,True
40,private,org.jitsi.videobridge.cc.BitrateController$Tra...,getTargetBitrate,,long,False,True,False,False,False,True,False,False,False,True
46,private,org.jitsi.videobridge.cc.BandwidthProbing,getSenderSSRC,,long,False,True,False,False,False,True,False,False,False,True
64,NaN,org.jitsi.videobridge.cc.vp8.VP8FrameProjection,isFullyProjected,long,boolean,False,False,False,False,True,True,False,False,False,True
67,public,org.jitsi.videobridge.rest.ColibriWebSocketSer...,getColibriWebSocketUrl,"java.lang.String, java.lang.String, java.lang....",java.lang.String,False,False,True,False,True,True,False,True,False,False
75,protected,org.jitsi.videobridge.octo.OctoEndpointMessage...,getId,java.lang.Object,java.lang.String,False,False,True,False,True,True,False,True,False,False
90,NaN,org.jitsi.videobridge.cc.vp8.VP8Frame,matchesSSRC,org.jitsi.videobridge.cc.vp8.VP8Frame,boolean,False,False,False,False,True,True,False,False,False,True
99,private,org.jitsi.videobridge.cc.BitrateController,getAvailableBandwidth,long,long,True,False,True,False,True,True,False,True,False,True


In [36]:
sorted(instrumentation_candidates_df['parent-FQN'].unique())

['org.jitsi.videobridge.Conference$NoOpDiagnosticContext',
 'org.jitsi.videobridge.Conference$NoOpTimeSeriesPoint',
 'org.jitsi.videobridge.cc.BandwidthProbing',
 'org.jitsi.videobridge.cc.BitrateController',
 'org.jitsi.videobridge.cc.BitrateController$TrackBitrateAllocation',
 'org.jitsi.videobridge.cc.GenericAdaptiveTrackProjectionContext',
 'org.jitsi.videobridge.cc.vp8.VP8Frame',
 'org.jitsi.videobridge.cc.vp8.VP8FrameProjection',
 'org.jitsi.videobridge.octo.OctoEndpointMessageTransport',
 'org.jitsi.videobridge.rest.ColibriWebSocketService']